In [3]:
import pandas as pd
import functions as f
import mypreprocess as p
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [4]:
data = pd.read_csv("./data/rossmann.csv")

In [5]:
df = f.clean_data(data)

In [6]:
df_red = df[[
            # 'Date', 
            # 'Store', 
            # 'DayOfWeek', 
            'Sales', 
            'Customers', 
            # 'Open', 
            # 'Promo',
            # 'StateHoliday', 
            # 'SchoolHoliday', 
            'StoreType', 
            'Assortment',
            'CompetitionDistance', 
            # 'Promo2', 
            # 'Promo2SinceWeek', 
            # 'Promo2SinceYear',
            # 'PromoInterval', 
            'Competition_Since_X_months']
        ]

In [7]:
df_red.columns

Index(['Sales', 'Customers', 'StoreType', 'Assortment', 'CompetitionDistance',
       'Competition_Since_X_months'],
      dtype='object')

In [8]:
from sklearn.model_selection import train_test_split

X_train = df_red.drop(["Sales"], axis=1)
y_train = df_red["Sales"]  # set the target column

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [9]:
X_train.columns

Index(['Customers', 'StoreType', 'Assortment', 'CompetitionDistance',
       'Competition_Since_X_months'],
      dtype='object')

In [10]:
preprocess = p.create_preprocessing()

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

linreg_pipe = make_pipeline(preprocess,
                            LinearRegression())

linreg_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['StoreType', 'Assortment']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Customers',
                                                   'Competition_Since_X_months',
                                                   'CompetitionDistance'])])),
                ('linearregression', LinearRegression())])

In [12]:
# linreg_pipe.named_steps['linearregression'].coef_
linreg_pipe.score(X_test, y_test)

0.8384716133317386

In [13]:
##### Save Model for pipeline.py
import pickle

filename = './pipeline/lr_pipeline.pkl'
pickle.dump(linreg_pipe, open(filename, 'wb'))